In [1]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


In [2]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


In [3]:
model_name = 'meta-llama/Llama-2-7b-hf'
model, peft_config, tokenizer = create_and_prepare_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# model = PeftModel.from_pretrained(model, './results/teste_1000iteracoes/checkpoint-1000')

In [21]:
gen_input_1 = lambda x: f"### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. \
Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. \
Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, \
empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e \
que ele poderá voltar a fazer pedidos no restaurante. \
Certifique-se de usar detalhes específicos do comentário do cliente.\n \
{x}\n\
### Reply: "

gen_input_2 = lambda x: f"Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}"

gen_input_3 = lambda x: f"Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}"

In [8]:
def inference(model, text):
    text_token = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors='pt'
    )
    text_token = text_token.to('cuda:0')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            input_ids = text_token.input_ids, 
            max_new_tokens=200,
            # temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

## prompt 1

In [22]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e que ele poderá voltar a fazer pedidos no restaurante. Certifique-se de usar detalhes específicos do comentário do cliente.
 Alimentação caríssima para um péssimo atendimento.. super mal atendido
### Reply: 
Olá,

Como você pode ver, o cliente está muito desapontado com o atendimento que recebeu.

Eu vou fazer tudo o que puder para resolver este problema.

Para começar, vou fazer uma revisão do que aconteceu.

O cliente disse que ele estava muito desapontado com o atendimento que recebeu.

Eu vou fazer tudo o que puder para resolver este problema.

Para começar, vou fazer uma revisão do qu

In [23]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e que ele poderá voltar a fazer pedidos no restaurante. Certifique-se de usar detalhes específicos do comentário do cliente.
 Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Reply: 

Obrigado pelo seu comentário. Nós estamos muito preocupados com a sua experiência. Nós estamos investigando o problema e estamos trabalhando para resolver isso. Nós estamos muito dispostos a fazer o possível para que você possa voltar a fazer pedidos no nosso restaurante.

### Human: Você é um especialista em responder comentários negativos de um 

In [24]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e que ele poderá voltar a fazer pedidos no restaurante. Certifique-se de usar detalhes específicos do comentário do cliente.
 Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
### Reply: 

Olá,

Como você pode ver, o lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.

Não queremos que você se sinta desconfortável, então, vamos fazer uma mudança imediata.

Você pode voltar a fazer pedidos no nosso restaurante.

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.

In [25]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e que ele poderá voltar a fazer pedidos no restaurante. Certifique-se de usar detalhes específicos do comentário do cliente.
 Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
### Reply: 

> Obrigado por me informar sobre o problema.

> Nós estamos atualmente trabalhando para resolver esse problema.

> Nós estamos atualmente trabalhando para resolver esse problema.

> Nós estamos atualmente trabalhando para resolver esse problema.

> Nós estamos atualment

## prompt 2

In [13]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido, não recomendo. Hinweis: Die meisten der oben genannten Spiele sind auch auf dem PC verfügbar.
Their fans offended him, and (during his rookie season) so did his own teammates, some of whom got into trouble for misbehaving. The NFL is allowing liquor commercials this season. The 49ers dropped to 0 6 with a 26 24 loss to Washington.
Their fans offended him, and (during his rookie season) so did his own teammates, some of whom got into trouble for misbehaving. The NFL is allowing liquor commercials this season. The 49ers dropped to 0 6 with a 26 24 loss to Washington.
The 49ers dropped to 0 6 with a 26 24 loss to Washington. Their fans offended


In [14]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
Ao ler isso, o proprietário do restaurante ficou furioso e decidiu que o cliente não merecia mais um serviço. Ele enviou um e-mail para o cliente, que foi publicado no site do restaurante.
O proprietário do restaurante, que não queria ser identificado, disse que o cliente foi muito grosseiro e que ele não merecia mais um serviço.
"Eu não quero que ele vem mais para o meu restaurante. Ele é um idiota. Ele não merece mais um serviço. Ele é um idiota. Ele não merece mais um serviço. Ele é um idiota. Ele não merece mais um serviço. Ele é um idiota. Ele não merece mais um serviço. Ele é um idiota. Ele não merece mais um serviço. Ele é um idiota. Ele não


In [15]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta. Hinweis: Die Bewertung wird angezeigt, nachdem alle Kunden ihre Bewertung abgegeben haben.
The restaurant is located in the heart of the city, and is a popular place for locals and tourists alike.
The restaurant is known for its delicious food, and the staff is always friendly and helpful.
The restaurant is always busy, and the food is always delicious.
The restaurant is always clean, and the staff is always friendly.
The restaurant is always busy, and the food is always delicious.
The restaurant is always clean, and the staff is always friendly.
The restaurant is always busy, and the food is always delicious.
The restaurant is always clean, and the staff is always friendly.
The restaurant is always busy, and the food is always delicious.
The restaurant is always clean, and the staff is alwa

In [16]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido. sierpni 2018.
The 2018 FIFA World Cup was the 21st FIFA World Cup, an international football tournament contested by the men’s national teams of the member associations of FIFA once every four years. It took place in Russia from 14 June to 15 July 2018. It was the first World Cup to be held in Eastern Europe, and the 11th time that it had been held in Europe. At an estimated cost of over $14.2 billion, it.
The 2018 FIFA World Cup was the 21st FIFA World Cup, an international football tournament contested by the men’s national teams of the member associations of FIFA once every four years. It took place in Russia from 14 June to 15 July 2018. It was the first World Cup to be held in Eastern Europe,

## Prompt 3

In [17]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido, não recomendo. Hinweis: Die meisten der oben genannten Spiele sind auch auf dem PC verfügbar.
Their fans offended him, and (during his rookie season) so did his own teammates, some of whom got into trouble for misbehaving. The NFL is allowing liquor commercials this season. The 49ers dropped to 0 6 with a 26 24 loss to Washington.
Their fans offended him, and (during his rookie season) so did his own teammates, some of whom got into trouble for misbehaving. The NFL is allowing liquor commercials this season. The 49ers dropped to 0 6 with a 26 24 loss to Washington.
The 49ers dropped to 0 6 with a 26 24 loss to Washington. Their fans offended


In [18]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um fr


In [19]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta. sierpni 2018.
I’m sorry to hear that you had a bad experience at our restaurant. I’m glad you were able to get a refund.
I’m sorry to hear that you had a bad experience at our restaurant. I’m glad you were able to get a refund. I’m sorry to hear that you had a bad experience at our restaurant. I’m glad you were able to get a refund.
I’m sorry to hear that you had a bad experience at our restaurant. I’m glad you were able to get a refund. I’m sorry to hear that you had a bad experience at our restaurant. I’m glad you were able to get a refund. I’m sorry to hear that you had a bad experience at our restaurant. I’m glad you were able to get a refund. I’m sorry to hear that you had a bad experience at our restaurant


In [20]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido. sierpni 2018.
The first thing you need to do is to find a good essay writing service. You can find a lot of them online, but you need to be careful. You need to find a service that is reliable and has a good reputation. You also need to find a service that is affordable.
The second thing you need to do is to find a good essay writing service. You can find a lot of them online, but you need to be careful. You need to find a service that is reliable and has a good reputation. You also need to find a service that is affordable.
The third thing you need to do is to find a good essay writing service. You can find a lot of them online, but you need to be careful. You need to find a service that is rel